In [174]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PC
from pinecone import ServerlessSpec
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob
from google.colab import userdata
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import FileSystemBlobLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import PyPDFParser
import time

In [ ]:
#gerar n versoes da pergunta
#criar novos docs a partir do artigo, categorizados

In [28]:
loader = DirectoryLoader(
    "/content/data",
    glob="*.pdf",
    loader_cls=PyPDFLoader
)

In [31]:
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Tamanho máximo de cada pedaço
    chunk_overlap=50 # Sobreposição entre pedaços
)
docs_splitted = text_splitter.split_documents(docs)

texts = [doc.page_content for doc in docs_splitted]


In [120]:

os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')


In [39]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(texts)

In [181]:
index_name = "rag"
pc = PC(api_key="pcsk_6JsENF_EkUKdhWBGJXSsKsbKdU2E5HAuGGq8AvfAvaCjFYj2BJUBxgEPq9GHQYMk2wK6ST")
index = pc.Index(index_name)

In [182]:
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


vectorstore = PineconeVectorStore.from_documents(docs_splitted, embedder, index_name=index_name)


In [193]:
query = "trap rf"
similar_docs = vectorstore.similarity_search(query)

In [194]:
similar_docs

[Document(id='db720f0b-91b1-4a59-a6d1-f6e9ebeb24ab', metadata={'author': '', 'creationdate': '2020-08-04T00:04:15+00:00', 'creator': 'LaTeX with hyperref package', 'keywords': '', 'moddate': '2020-08-04T00:04:15+00:00', 'page': 8.0, 'page_label': '9', 'producer': 'pdfTeX-1.40.17', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'source': '/content/data/2008.00065v1.pdf', 'subject': '', 'title': '', 'total_pages': 15.0, 'trapped': '/False'}, page_content='removed using a 7th-order Bessel notch ﬁlter after the\noutput ampliﬁers, which provides 40 dB insertion loss at\nthe trap rf frequency, with a 3 dB bandwidth of 14 MHz.\nA ﬁnal dissipative Gaussian lowpass ﬁlter withf3dB = 25\nMHz gives an additional 18 dB insertion loss at the trap\nrf frequency, while also reducing the noise bandwidth.\nThis improves pulse discrimination, which is carried out'),
 Document(id='80f96859-43a5-410d-aa95-bd84ca5f2cd5', metadata={'author': '', 'c

In [165]:
!pip install -qU langchain-pinecone
